<a href="https://colab.research.google.com/github/Samarpeet/OCR-for-Sanskrit/blob/main/REDO_FULL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 !pip install -q kaggle

In [ ]:
# from google.colab import files
# files.upload()

In [ ]:
!mkdir ~/.kaggle

In [ ]:
!cp kaggle.json ~/.kaggle/

cp: cannot stat 'kaggle.json': No such file or directory


In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
!kaggle datasets list

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 403, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [ ]:
!kaggle datasets download -d anurags397/hindi-mnist-data

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 403, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [ ]:
!unzip hindi-mnist-data.zip

unzip:  cannot find or open hindi-mnist-data.zip, hindi-mnist-data.zip.zip or hindi-mnist-data.zip.ZIP.


In [ ]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
import matplotlib.pyplot as plt


## Importing Necessary Libraries

In [ ]:
import torch
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader,random_split,TensorDataset
from torchvision.utils import make_grid
from torchvision.transforms import ToTensor
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
import torchvision.models as models
import matplotlib.pyplot as plt

In [ ]:
from tqdm.notebook import tqdm
from tqdm import tqdm

In [ ]:
project_name = 'sanskrit classification'

In [ ]:
path_folder = "/content/DevanagariHandwrittenDigitDataset/Train"
os.listdir(path_folder)

FileNotFoundError: ignored

## Number of unique characters

In [ ]:
unique_nos = []
cl = os.listdir(path_folder)
for i in cl:
    x = i.split('_')
    if x[0] not in unique_nos:
        unique_nos.append(x[0])
print("Number of Unique Characters : ",len(unique_nos))
print("Unique Characters: ",unique_nos)

## Loading Training and Test Dataset as Tensor

In [ ]:
transform = transforms.Compose(
    [transforms.Resize((32, 32)),
     transforms.ToTensor()])

In [ ]:
transform_new = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor()])


mean = [0.5, 0.5, 0.5]
std = [0.5, 0.5, 0.5]

# Compose the transformations
transform_new_2 = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
#     transforms.Normalize(mean=mean, std=std),
    transforms.RandomRotation(degrees=15),  # Add random rotation (adjust degrees as needed)
    transforms.RandomCrop((32, 32)),  # Add random crop (adjust size as needed)
    transforms.GaussianBlur(kernel_size=3)  # Add Gaussian blur (adjust kernel_size as needed)
])

In [ ]:
dataset = ImageFolder("/content/DevanagariHandwrittenDigitDataset/Train",transform=transform_new)
test_ds = ImageFolder("/content/DevanagariHandwrittenDigitDataset/Test",transform=transform_new)

In [ ]:
dataset

In [ ]:
print("Number of training images: ",len(dataset))
print("Number of testing images: ",len(test_ds))

### Number of classes

In [ ]:
num_classes = dataset.classes
print("Number of classes: ",len(num_classes))
print(num_classes)

### Visualising a single image

In [ ]:
image, label  = dataset[0]
print("Image shape:",image.shape)
print("Image Label: ",label)
print("Image Label: ",dataset.classes[label])
print(image)

In [ ]:
image, label  = dataset[0]
fig,(ax1,ax2) = plt.subplots(figsize=(15,5),nrows=1,ncols=2)
ax1.imshow(image.permute(1,2,0))
ax1.set_title("original image")
ax2.imshow(1-image.permute(1,2,0))
ax2.set_title("inverted image")
plt.show()

# Validation Dataset and Dataloader

In [ ]:
random_seed = 42
torch.manual_seed(random_seed)

In [ ]:
validation_split = 0.3
val_size = int(len(dataset) * validation_split)
train_size = len(dataset) - val_size

train_ds,val_ds = random_split(dataset,[train_size,val_size])

In [ ]:
# batch_size = 128
batch_size = 32

train_loader = DataLoader(train_ds,batch_size=batch_size,num_workers=2,shuffle=True, pin_memory=True)
val_loader = DataLoader(val_ds,batch_size=batch_size,num_workers=2,shuffle=True, pin_memory=True)
test_loader = DataLoader(test_ds,batch_size=batch_size,num_workers=2,shuffle=True, pin_memory=True)

## Visualising a Batch of images

In [ ]:
import itertools

# Limit the number of batches processed for testing
subset_loader = itertools.islice(train_loader, 5)  # Change '5' to the desired number of batches

for batch in subset_loader:
    for image in batch:
        # Check the shape of each image
        print(image.shape)

# Building the model

### Building a Base Image Classification Model

In [ ]:
def accuracy(outputs, labels):
  _, preds = torch.max(outputs, dim=1)
  return torch.tensor(torch.sum(preds == labels).item() / len(preds))


class ImageClassificationBase(nn.Module): #nn.Module, which is the base class for all PyTorch models.

  def training_step(self,batch):
    images,labels = batch
    out = self(images)
    loss = F.cross_entropy(out,labels)
    return loss

  def validation_step(self,batch):
    images,labels = batch
    out = self(images)
    loss = F.cross_entropy(out,labels)
    acc = accuracy(out,labels)
    return {'val_loss':loss,'val_acc':acc}

  def validation_epoch_end(self,outputs):
    batch_loss = [out['val_loss'] for out in outputs]
    epoch_loss = torch.stack(batch_loss).mean()
    batch_acc = [out['val_acc'] for out in outputs]
    epoch_acc = torch.stack(batch_acc).mean()
    return {'val_loss':epoch_loss.item(),'val_acc':epoch_acc.item()}

  def epoch_end(self,epoch,result):
    print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(epoch, result['train_loss'], result['val_loss'], result['val_acc']))

## Buiding a CNN model

In [ ]:
from sklearn.metrics import accuracy_score
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

class Sanskrit_characters_Model_cnn(nn.Module):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.AdaptiveAvgPool2d(1)
        )

        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 10)  # Assuming 10 classes
        )

    def forward(self, x):
        out = self.network(x)
        out = self.fc(out)
        return out

    def training_step(self, batch):
        images, labels = batch
        outputs = self(images)
        loss = F.cross_entropy(outputs, labels)
        return loss

    def validation_step(self, batch):
        images, labels = batch
        outputs = self(images)
        loss = F.cross_entropy(outputs, labels)
        _, preds = torch.max(outputs.cpu(), dim=1)  # Ensure predictions are on CPU
        return {'val_loss': loss, 'labels': labels, 'preds': preds}

    def validation_epoch_end(self, outputs):
        # Extract losses and labels
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.tensor(batch_losses).mean()  # Calculate mean loss

        # Extract labels and predictions
        all_labels = torch.cat([x['labels'] for x in outputs])
        all_preds = torch.cat([x['preds'] for x in outputs])

        # Calculate accuracy
        val_acc = accuracy_score(all_labels.cpu().numpy(), all_preds.cpu().numpy())

        return {'val_loss': epoch_loss.item(), 'val_acc': val_acc}

    def epoch_end(self, epoch, result):
        print(f"Epoch [{epoch+1}], train_loss: {result['train_loss']:.4f}, val_loss: {result['val_loss']:.4f}, val_acc: {result['val_acc']:.4f}")


## Building a VGG16 model using Transfer Learning

In [ ]:
class Sanskrit_characters_Model_VGG16(ImageClassificationBase):

  def __init__(self):
    super().__init__()
    self.network = models.vgg16(pretrained=True)
    num_ftrs = self.network.classifier[6].in_features
    # Change the fully connected layer for 10 classes
    self.network.classifier[6] = nn.Linear(num_ftrs, 10)

  def forward(self, xb):
    out = self.network(xb)
    return out

## Building a resnet34 model using Transfer Learning

In [ ]:
class Sanskrit_characters_Model2(ImageClassificationBase):

  def __init__(self):
    super().__init__()
    self.network = models.resnet34(pretrained=True)
    num_ftrs = self.network.fc.in_features
    self.network.fc = nn.Linear(num_ftrs, 10)

  def forward(self,xb):
    out = self.network(xb)
    return out

# Training and Evaluation

In [ ]:
@torch.no_grad()
def evaluate(model,val_loader):
  model.eval()
  outputs = [model.validation_step(batch) for batch in val_loader]
  return model.validation_epoch_end(outputs)

@torch.no_grad()
def evaluate_cnn(model, val_loader):
    model.eval()
    outputs = []
    for batch in val_loader:
        images, labels = batch
        # Move the batch to the appropriate device (CPU or GPU)
        images = images.to('cuda')  # Move the images to the GPU
        labels = labels.to('cuda')  # Move the labels to the GPU
        output = model(images)
        loss = F.cross_entropy(output, labels)
        _, preds = torch.max(output, dim=1)
        outputs.append({'val_loss': loss.item(), 'labels': labels.cpu(), 'preds': preds.cpu()})
    return model.validation_epoch_end(outputs)

In [ ]:
def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        model.train()
        train_losses = []
        for batch in tqdm(train_loader):
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        model.epoch_end(epoch, result)  # Passing the correct arguments
        history.append(result)
    return history

# Using GPU

In [ ]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device

    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl:
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [ ]:
device = get_default_device()
device

In [ ]:
train_loader = DeviceDataLoader(train_loader, device)
val_loader = DeviceDataLoader(val_loader, device)
test_loader = DeviceDataLoader(test_loader, device)

In [ ]:
model = to_device(Sanskrit_characters_Model2(), device)

In [ ]:
model

### Training the model

In [ ]:
evaluate(model,val_loader)

In [ ]:
 model_cnn = Sanskrit_characters_Model_cnn()
 model_cnn = model_cnn.to('cuda')

In [ ]:
 evaluate_cnn(model_cnn,val_loader)

#### VGG 16 model

In [ ]:
model_vgg16 = to_device(Sanskrit_characters_Model_VGG16(), device)

model_vgg16

In [ ]:
evaluate(model_vgg16,val_loader)

#### resnet model running

In [ ]:
history = fit(10, 0.001, model, train_loader, val_loader, opt_func = torch.optim.Adam)

#### cnn model running

In [ ]:
history_cnn = fit(10, 0.001, model_cnn, train_loader, val_loader, opt_func = torch.optim.Adam)

#### vgg16 model running

In [ ]:
history_vgg16 = fit(10, 0.001, model_vgg16, train_loader, val_loader, opt_func = torch.optim.Adam)

In [ ]:
torch.save(model.state_dict(), 'SK-resnet.pth')

torch.save(model_cnn.state_dict(), 'SK-cnn.pth')

torch.save(model_vgg16.state_dict(), 'SK-vgg16.pth')

Plotting Accuracy and losses

In [ ]:
def plot_losses(history):
    train_losses = [x.get('train_loss') for x in history]
    val_losses = [x['val_loss'] for x in history]
    plt.plot(train_losses, '-bx')
    plt.plot(val_losses, '-rx')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(['Training', 'Validation'])
    plt.title('Loss vs. No. of epochs')

In [ ]:
def plot_accuracies(history):
    accuracies = [x['val_acc'] for x in history]
    plt.plot(accuracies, '-x')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.title('Accuracy vs. No. of epochs')

In [ ]:
plot_accuracies(history)

plot_accuracies(history_cnn)

plot_accuracies(history_vgg16)

In [ ]:
plot_losses(history)

In [ ]:
plot_losses(history_cnn)

In [ ]:
plot_losses(history_vgg16)

In [ ]:
evaluate(model,val_loader)

In [ ]:
evaluate_cnn(model_cnn,val_loader)

In [ ]:
evaluate(model_vgg16,val_loader)

## Evaluation and Prediction on Test Data

In [ ]:
x = evaluate(model,test_loader)
x

In [ ]:
 x = evaluate_cnn(model_cnn,test_loader)
 x

In [ ]:
x = evaluate(model_vgg16,test_loader)
x

## Prediction on some single image of test data

In [ ]:
def predict_image(img, model):
    xb = to_device(img.unsqueeze(0), device)
    yb = model(xb)
    _, preds  = torch.max(yb, dim=1)
    return dataset.classes[preds[0].item()]

def predict_image_cnn(img, model):
    # Move the input image tensor to the appropriate device (CPU or GPU)
    xb = to_device(img.unsqueeze(0), device)
    # Ensure the model and input tensor are on the same device
    model = model.to(device)
    yb = model(xb)
    _, preds = torch.max(yb, dim=1)
    return dataset.classes[preds[0].item()]

In [ ]:
img, label = test_ds[1300]
plt.imshow(img.permute(1, 2, 0))
print('    Label:', dataset.classes[label])
print('Predicted:', predict_image(img, model))
print('--------')

img, label = test_ds[1300]
plt.imshow(img.permute(1, 2, 0))
print('    Label:', dataset.classes[label])
print('Predicted:', predict_image(img, model_vgg16))
print('--------')

img, label = test_ds[1300]
plt.imshow(img.permute(1, 2, 0))
print('    Label:', dataset.classes[label])
print('Predicted:', predict_image_cnn(img, model_cnn))


In [ ]:
img, label = test_ds[980]
plt.imshow(img.permute(1, 2, 0))
print('Label:', dataset.classes[label], ', Predicted:', predict_image(img, model))

img, label = test_ds[980]
plt.imshow(img.permute(1, 2, 0))
print('Label:', dataset.classes[label], ', Predicted:', predict_image(img, model_vgg16))

img, label = test_ds[980]
plt.imshow(img.permute(1, 2, 0))
print('Label:', dataset.classes[label], ', Predicted:', predict_image_cnn(img, model_cnn))

In [ ]:
img, label = test_ds[254]
plt.imshow(img.permute(1, 2, 0))
print('Label:', dataset.classes[label], ', Predicted:', predict_image(img, model))

img, label = test_ds[254]
plt.imshow(img.permute(1, 2, 0))
print('Label:', dataset.classes[label], ', Predicted:', predict_image(img, model_vgg16))

img, label = test_ds[254]
plt.imshow(img.permute(1, 2, 0))
print('Label:', dataset.classes[label], ', Predicted:', predict_image_cnn(img, model_cnn))

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score, classification_report

# Assuming `model` is your trained model
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in test_loader:  # Assuming you want to evaluate on the test dataset
        images, labels = batch
        outputs = model(images)
        _, preds = torch.max(outputs, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Compute the F1 score, precision, and recall
f1 = f1_score(all_labels, all_preds, average='weighted')
precision = precision_score(all_labels, all_preds, average='weighted')
recall = recall_score(all_labels, all_preds, average='weighted')

# Display the results
print(f"F1 Score: {f1:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")

# # If you want a detailed report by class
# class_names = dataset.classes  # Use the class names from your dataset
# report = classification_report(all_labels, all_preds, target_names=class_names)
# print(report)
